In [6]:
# ===================================================================
# ЭТАП 2: Подготовка данных и инжиниринг признаков
# ===================================================================

# Импортируем pandas для работы с таблицами и datetime для работы с датами
import pandas as pd
from datetime import datetime
import os

print("--- Начало Этапа 2: Предобработка данных ---")

# Загружаем исходный датасет. 
# На этом этапе мы работаем с "сырыми" данными, загруженными после EDA.
try:
    df = pd.read_csv('CarRentalDataV1.csv')
    print(f"✅ Исходный датасет 'CarRentalDataV1.csv' успешно загружен. Размер: {df.shape}")
except FileNotFoundError:
    print("❌ Ошибка: Файл 'CarRentalDataV1.csv' не найден.")
    # Прерываем выполнение, если основной файл отсутствует
    # В ноутбуке можно просто остановить выполнение ячейки

# Создаем копию датафрейма для всех преобразований, чтобы не изменять исходный df
df_processed = df.copy()

--- Начало Этапа 2: Предобработка данных ---
✅ Исходный датасет 'CarRentalDataV1.csv' успешно загружен. Размер: (5851, 16)


### Шаг 2.1: Очистка данных (Data Cleaning)

На этом шаге мы приводим данные в порядок:
1.  **Обрабатываем пропущенные значения (NaN):**
    - Для числовых счетчиков (`rating`, `reviewCount`, `renterTripsTaken`) предполагаем, что пропуск означает отсутствие данных (например, у новой машины нет поездок), поэтому заменяем их на `0`.
    - Для категориальных признаков (`fuelType`, `vehicle.type`) заменяем редкие пропуски самым частым значением (модой).
2.  **Исправляем типы данных:** Идентификаторы и счетчики преобразуем в целочисленный тип (`int`).
3.  **Удаляем ненужные столбцы:** `location.country` (везде 'US') и `airportcity` (часто дублирует `location.city`) не несут уникальной информации для модели.

In [7]:
# --- 1. Обработка пропущенных значений ---
for col in ['rating', 'reviewCount', 'renterTripsTaken']:
    if df_processed[col].isnull().sum() > 0:
        df_processed[col] = df_processed[col].fillna(0)
print("Пропуски в числовых счетчиках заменены на 0.")

for col in ['fuelType', 'vehicle.type', 'location.city', 'location.state']:
    if df_processed[col].isnull().sum() > 0:
        mode_value = df_processed[col].mode()[0]
        df_processed[col] = df_processed[col].fillna(mode_value)
print("Пропуски в категориальных признаках заменены модой.")

# --- 2. Исправление типов данных ---
df_processed['owner.id'] = df_processed['owner.id'].astype(int)
df_processed['renterTripsTaken'] = df_processed['renterTripsTaken'].astype(int)
df_processed['reviewCount'] = df_processed['reviewCount'].astype(int)
print("Типы данных для id, trips и reviews исправлены на integer.")

# --- 3. Удаление ненужных столбцов и дубликатов ---
columns_to_drop = ['location.country', 'airportcity']
df_processed = df_processed.drop(columns=columns_to_drop, errors='ignore')
print(f"Колонки {columns_to_drop} удалены.")

if df_processed.duplicated().sum() > 0:
    df_processed = df_processed.drop_duplicates(inplace=True)
    print(f"Удалено {df_processed.duplicated().sum()} дублирующихся строк.")

print("\n✅ Очистка данных завершена.")
print(f"Размер датасета после очистки: {df_processed.shape}")

Пропуски в числовых счетчиках заменены на 0.
Пропуски в категориальных признаках заменены модой.
Типы данных для id, trips и reviews исправлены на integer.
Колонки ['location.country', 'airportcity'] удалены.

✅ Очистка данных завершена.
Размер датасета после очистки: (5851, 14)


### Шаг 2.2: Инжиниринг признаков (Feature Engineering)

Это самый творческий и важный шаг подготовки. Мы создаем новые, более информативные признаки, которые помогут модели лучше понять "контекст" каждой машины.
1.  **`vehicle.age`**: Возраст автомобиля. Интуитивно понятно, что год выпуска влияет на спрос.
2.  **Агрегированные признаки:** Для каждого автомобиля мы вычисляем характеристики рынка, на котором он находится:
    - `city_avg_trips`: Средний спрос в городе.
    - `city_car_count`: Уровень конкуренции в городе.
    - `city_avg_rate`: Средний уровень цен в городе.
    
Эти "контекстуальные" признаки являются очень мощными предикторами.

In [8]:
print("\n--- Создание новых признаков ---")

# --- 1. Создание признака "Возраст автомобиля" ---
current_year = 2023 # Используем фиксированный год для воспроизводимости
df_processed['vehicle.age'] = current_year - df_processed['vehicle.year']
# Удаляем возможные аномалии (машины из будущего)
df_processed = df_processed[df_processed['vehicle.age'] >= 0]
print("Создан признак 'vehicle.age'.")

# --- 2. Создание агрегированных признаков по географии ---
# Сначала считаем статистики по каждому городу и штату
city_stats = df_processed.groupby('location.city').agg(
    city_avg_trips=('renterTripsTaken', 'mean'),
    city_car_count=('owner.id', 'nunique'),
    city_avg_rate=('rate.daily', 'mean')
).reset_index()

state_stats = df_processed.groupby('location.state').agg(
    state_avg_trips=('renterTripsTaken', 'mean'),
    state_car_count=('owner.id', 'nunique')
).reset_index()

# Теперь присоединяем эти статистики к основному датафрейму
df_processed = pd.merge(df_processed, city_stats, on='location.city', how='left')
df_processed = pd.merge(df_processed, state_stats, on='location.state', how='left')
print("Созданы и присоединены агрегированные признаки по городу и штату.")

print("\n✅ Инжиниринг признаков завершен.")
print("Пример данных с новыми признаками:")
display(df_processed[['renterTripsTaken', 'vehicle.age', 'city_avg_trips', 'city_car_count']].head())


--- Создание новых признаков ---
Создан признак 'vehicle.age'.
Созданы и присоединены агрегированные признаки по городу и штату.

✅ Инжиниринг признаков завершен.
Пример данных с новыми признаками:


,renterTripsTaken,vehicle.age,city_avg_trips,city_car_count
0,13,4.0,78.117647,15
1,2,5.0,2.000000,1
2,28,11.0,19.045455,18
3,21,5.0,19.045455,18
4,3,13.0,19.045455,18


### Шаг 2.3 и 2.4: Кодирование, разделение и сохранение

Последние шаги перед моделированием:
1.  **Выбор признаков:** Отбираем столбцы, которые будем использовать для обучения.
2.  **Кодирование категорий:** Преобразуем текстовые признаки (`fuelType`, `vehicle.type`) в числовой формат с помощью **One-Hot Encoding**.
3.  **Разделение выборки:** Делим данные на обучающую (`train`) и тестовую (`test`) части.
4.  **Сохранение результатов:** Сохраняем подготовленные датасеты в `.csv` файлы. Это лучшая практика, которая делает наш проект модульным и избавляет от проблем с переменными на следующих этапах.

In [9]:
from sklearn.model_selection import train_test_split

# --- 1. Выбор признаков и кодирование ---
print("\n--- Подготовка финального датафрейма для модели ---")
target = 'renterTripsTaken'
numerical_features = [
    'rating', 'reviewCount', 'rate.daily', 'vehicle.age',
    'city_avg_trips', 'city_car_count', 'city_avg_rate',
    'state_avg_trips', 'state_car_count'
]
categorical_features = ['fuelType', 'vehicle.type']

# Создаем датафрейм только с нужными колонками
model_df = df_processed[numerical_features + categorical_features + [target]].copy()

# Применяем One-Hot Encoding
model_df = pd.get_dummies(model_df, columns=categorical_features, prefix=categorical_features, drop_first=True)

# Обработка возможных пропусков после merge (например, для новых городов, которых не было в train)
if model_df.isnull().sum().sum() > 0:
    model_df = model_df.fillna(model_df.median())
print("Данные закодированы и готовы к разделению.")

# --- 2. Разделение данных ---
X = model_df.drop(columns=[target])
y = model_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Данные разделены на обучающую и тестовую выборки.")

# --- 3. Сохранение результатов на диск ---
output_dir = 'processed_data'
os.makedirs(output_dir, exist_ok=True)

# Сохраняем все подготовленные части
X_train.to_csv(os.path.join(output_dir, 'X_train.csv'), index=False)
X_test.to_csv(os.path.join(output_dir, 'X_test.csv'), index=False)
y_train.to_csv(os.path.join(output_dir, 'y_train.csv'), index=False)
y_test.to_csv(os.path.join(output_dir, 'y_test.csv'), index=False)
# Сохраняем полный model_df для гибкости на этапе 3
model_df.to_csv(os.path.join(output_dir, 'model_df.csv'), index=False)
# Сохраняем city_stats, он понадобится для этапа 4
city_stats.to_csv(os.path.join(output_dir, 'city_stats.csv'), index=False)

print(f"\n✅ Все подготовленные датасеты сохранены в папку '{output_dir}'.")
print(f"Размер X_train: {X_train.shape}, Размер X_test: {X_test.shape}")
print("\n--- Этап 2 полностью завершен. ---")


--- Подготовка финального датафрейма для модели ---
Данные закодированы и готовы к разделению.
Данные разделены на обучающую и тестовую выборки.

✅ Все подготовленные датасеты сохранены в папку 'processed_data'.
Размер X_train: (4680, 16), Размер X_test: (1171, 16)

--- Этап 2 полностью завершен. ---


In [ ]:
# --- Расчет порогов для бизнес-логики ---


demand_thresholds = {
    "average_demand": city_stats['city_avg_trips'].quantile(0.50),
    "high_demand": city_stats['city_avg_trips'].quantile(0.75)
}

print("Рассчитанные пороги спроса:")
print(demand_thresholds)

# Сохраняем пороги в простой текстовый файл (json)
import json
output_dir = 'processed_data' # Убедитесь, что эта папка существует
with open(os.path.join(output_dir, 'demand_thresholds.json'), 'w') as f:
    json.dump(demand_thresholds, f)

print(f"\n✅ Пороги успешно сохранены в файл '{output_dir}/demand_thresholds.json'.")

Рассчитанные пороги спроса:
{'average_demand': np.float64(15.0), 'high_demand': np.float64(30.52777777777778)}

✅ Пороги успешно сохранены в файл 'processed_data/demand_thresholds.json'.
